In [1]:
# 读取谷歌云盘的数据请求
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=False)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
# !cd '/content/drive/MyDrive'
import os
# # os.chdir('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis')
# # os.chdir('/content/drive')
# import torch '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF without DA/12k Drive End Bearing Fault Data'
datafiles = os.listdir('/content/drive/MyDrive/')
print(datafiles)

['Colab Notebooks', 'Experiment', 'Colab-Notebooks']


In [3]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
!pip install pyts --target=/content/drive/MyDrive/Colab-Notebooks/python_packages

In [8]:
!nvidia-smi

Tue Oct 18 15:39:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 测试两个type和两个severity，GAF/MTF，CNN

# focus on file '12k Drive End Bearing Fault Data'

import os, sys
import scipy.io as io
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pylab
from PIL import Image
from torch.autograd import Variable
from tqdm import tqdm
import logging
import time

plt.rc('font',family='Times New Roman')

# GPU
device='cuda:0'

# Test
# choice = 7
for choice in range(8,16):

  print("choice",choice)

  # Change the path here to test different data sources and data augmentation methods.
  # Simple
  path1 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/with DA/12k Drive End Bearing Fault Data'   
  path2 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/with DA/12k Fan End Bearing Fault Data'
  path3 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/with DA/48k Drive End Bearing Fault Data'
  # Hilbert
  path4 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/with DA/12k Drive End Bearing Fault Data'   
  path5 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/with DA/12k Fan End Bearing Fault Data'
  path6 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/with DA/48k Drive End Bearing Fault Data'
  # GAF/MTF/GAF+MTF
  path7 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Drive End Bearing Fault Data'
  path8 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Fan End Bearing Fault Data'
  path9 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/48k Drive End Bearing Fault Data'
  path10 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Drive End Bearing Fault Data'
  path11 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Fan End Bearing Fault Data'
  path12 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/48k Drive End Bearing Fault Data'
  path13 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Drive End Bearing Fault Data'
  path14 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Fan End Bearing Fault Data'
  path15 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/48k Drive End Bearing Fault Data'
  paths = [path1, path2, path3, path4, path5, path6, path7, path8, path9, path10, path11, path12, path13, path14, path15]
  path = paths[choice-1]

  # Edit the corresponding testtype.
  testtype1 = 'Ex1_12kDE_Simple'
  testtype2 = 'Ex1_12FE_Simple'
  testtype3 = 'Ex1_48kDE_Simple'
  testtype4 = 'Ex1_12kDE_Hilbert'
  testtype5 = 'Ex1_12FE_Hilbert'
  testtype6 = 'Ex1_48kDE_Hilbert'
  testtype7 = 'Ex1_12kDE_GAF'
  testtype8 = 'Ex1_12FE_GAF'
  testtype9 = 'Ex1_48kDE_GAF'
  testtype10 = 'Ex1_12kDE_MTF'
  testtype11 = 'Ex1_12FE_MTF'
  testtype12 = 'Ex1_48kDE_MTF'
  testtype13 = 'Ex1_12kDE_GAF_MTF'
  testtype14 = 'Ex1_12FE_GAF_MTF'
  testtype15 = 'Ex1_48kDE_GAF_MTF'
  testtypes= [testtype1, testtype2, testtype3, testtype4, testtype5, testtype6, testtype7, testtype8, testtype9, testtype10, testtype11, testtype12, testtype13, testtype14, testtype15]
  testtype = testtypes[choice-1]

  # Simple transform
  def simple(arr):
      # arr: 256, ans: 16*16
      Len = int(len(arr)**0.5)
      ans=[[0 for i in range(Len)] for j in range(Len)]
      k=1
      x,y=0,0
      for i in range(Len):
          for j in range(Len):
              if i%2==0:
                  ans[i][j]=arr[i*Len+j]
              else:
                  ans[i][j]=arr[i*Len+Len-1-j]
      return ans

  # Hilbet transform
  def hilbert(arr):

      def path(arr, a: int, pos):
          ans = [[0 for _ in range(a)] for _ in range(a)]
          if arr == []: return ans
          if a == 1:
              return [[arr[0]]]
          sub = a*a // 4
          for i in range(1,5):
              if sub * (i-1) > len(arr): break
              subans = path(arr[sub * (i-1): sub * i], a//2, i)
              for x in range(0, a//2):
                  for y in range(0, a//2):
                      if i == 1: ans[a - 1 - y][a//2 -1 - x] = subans[x][y]
                      if i == 2: ans[x][y] = subans[x][y]
                      if i == 3: ans[x][y + a//2] = subans[x][y]
                      if i == 4: ans[y + a//2][a//2 + x] = subans[x][y]
          return ans

      n = len(arr)
      a = 1
      while a ** 2 < n:
          a *= 2
      ans = path(arr, a, 2)
      return ans

  def gen_data():
      f={'Ball':0,'Inner':1,'Outer':2,'7':0,'14':1,'21':2,'28':3}

      datafiles = os.listdir(path)
      lentrain = int(len(datafiles) * 0.7)
      lentest = len(datafiles) - lentrain
      X, Y1, Y2=[], [], []
      for file in tqdm(datafiles):
          if choice>=1 and choice<=6:
              data = io.loadmat(os.path.join(path, file))['data']
              if choice<=3:
                  data = simple(data)
              else:
                  data = hilbert(data)
          else:
              dataGAF = io.loadmat(os.path.join(path, file))['GAF']
              dataMTF = io.loadmat(os.path.join(path, file))['MTF']
              if choice<=9:
                  data = dataGAF
              elif choice<=12:
                  data = dataMTF
              else:
                  data = [dataGAF, dataMTF]
          y1, y2, _ = file.split('_')
          Y1.append(f[y1])
          Y2.append(f[y2])
          X.append(data)

      if choice<=6:
          X = torch.tensor(np.array(X).reshape((len(X),1, 16, 16)))
      elif choice<=12:
          X = torch.tensor(np.array(X).reshape((len(X),1, 256, 256)))
      else:
          X = torch.tensor(np.array(X).reshape((len(X),2, 256, 256)))
      Y1 = torch.tensor(np.array(Y1).reshape((len(X), 1))).to(torch.long)
      Y2 = torch.tensor(np.array(Y2).reshape((len(X), 1))).to(torch.long)

      n=len(X)
      idx = torch.randperm(n)
      X_train = X[idx[:lentrain],]
      X_test = X[idx[lentrain:],]
      Y1_train = Y1[idx[:lentrain],]
      Y1_test = Y1[idx[lentrain:],]
      Y2_train = Y2[idx[:lentrain],]
      Y2_test = Y2[idx[lentrain:],]

      return X_train, Y1_train, Y2_train, X_test, Y1_test, Y2_test

  # For simple or hilbert
  class Net1(nn.Module):
      def __init__(self):
          super(Net1, self).__init__()

          self.conv = nn.Conv2d(1, 8, kernel_size=5, padding=2) 
          self.conv1 = nn.Conv2d(8, 16, kernel_size=3, padding=1) 
          self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1) 


          self.mp = nn.MaxPool2d(2)
          self.relu = nn.ReLU()
          self.tanh = nn.Tanh()
          self.fc1 = nn.Linear(16*4*4,1000)
          self.fc2 = nn.Linear(16*4*4,1000)
          self.fc3 = nn.Linear(1000,100)
          self.fc4 = nn.Linear(1000,100)
          self.fc5 = nn.Linear(100,3)
          self.fc6 = nn.Linear(100,3)
          self.sigmod=nn.Sigmoid()
          self.logsoftmax = nn.LogSoftmax()
          self.softmax = nn.Softmax(dim=-1)

      def forward(self, x):
          out = self.mp(self.conv(x))

          out1 = self.mp(self.conv1(out))
          out1 = out1.view(-1)
          out1 = self.sigmod(self.fc1(out1))
          out1 = self.sigmod(self.fc3(out1))
          out1 = self.sigmod(self.fc5(out1))

          out2 = self.mp(self.conv2(out))
          out2 = out2.view(-1)
          out2 = self.sigmod(self.fc2(out2))
          out2 = self.sigmod(self.fc4(out2))
          out2 = self.sigmod(self.fc6(out2))

          return out1, out2

  # For GAF/MTF/GAF-MTF
  class Net2(nn.Module):
      def __init__(self):
          super(Net2, self).__init__()

          channel = 1 if choice<=12 else 2
          self.conv = nn.Conv2d(channel, 8, kernel_size=15, padding=7) 
          self.conv1 = nn.Conv2d(8, 16, kernel_size=5, padding=2) 
          self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=2) 
          self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
          self.conv4 = nn.Conv2d(16, 32, kernel_size=3, padding=1) 

          self.mp = nn.MaxPool2d(2)
          self.relu = nn.ReLU()
          self.tanh = nn.Tanh()
          self.fc1 = nn.Linear(32*32*32,1000)
          self.fc2 = nn.Linear(32*32*32,1000)
          self.fc3 = nn.Linear(1000,100)
          self.fc4 = nn.Linear(1000,100)
          self.fc5 = nn.Linear(100,3)
          self.fc6 = nn.Linear(100,3)
          self.sigmod=nn.Sigmoid()
          self.logsoftmax = nn.LogSoftmax()
          self.softmax = nn.Softmax(dim=-1)

      def forward(self, x):
          out = self.mp(self.conv(x))

          out1 = self.mp(self.conv1(out))
          out1 = self.mp(self.conv3(out1))
          out1 = out1.view(-1)
          out1 = self.sigmod(self.fc1(out1))
          out1 = self.sigmod(self.fc3(out1))
          out1 = self.sigmod(self.fc5(out1))

          out2 = self.mp(self.conv2(out))
          out2 = self.mp(self.conv4(out2))
          out2 = out2.view(-1)
          out2 = self.sigmod(self.fc2(out2))
          out2 = self.sigmod(self.fc4(out2))
          out2 = self.sigmod(self.fc6(out2))

          return out1, out2

  X_train, Y1_train, Y2_train, X_test, Y1_test, Y2_test = gen_data()

  X_train=torch.tensor(X_train).to(device)
  Y1_train=torch.tensor(Y1_train).to(device)
  Y2_train=torch.tensor(Y2_train).to(device)
  X_test=torch.tensor(X_test).to(device)
  Y1_test=torch.tensor(Y1_test).to(device)
  Y2_test=torch.tensor(Y2_test).to(device)


  if choice<=6:
      model = Net1()
  else:
      model = Net2()
  model=model.to(device)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

  logs = str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))+'\tstart training!'

  Max_epoch = 40
  acclist = []
  for epoch in range(Max_epoch):
      print('*' * 25, 'epoch {}'.format(epoch + 1), '*' * 25)
      running_loss1 = 0.0
      running_loss2 = 0.0
      running_acc1_train = 0.0
      running_acc2_train = 0.0
      n = len(X_test)
      if epoch==0 or running_acc1_test*running_acc2_test==0.0:
        w1, w2 = 0.5, 0.5
      else:
        w1, w2 = (n-running_acc1_test)/(2*n-running_acc1_test-running_acc2_test), (n-running_acc2_test)/(2*n-running_acc1_test-running_acc2_test)

      running_acc1_test = 0.0
      running_acc2_test = 0.0

      for t,x in tqdm(enumerate(X_train)):
          data, target1, target2 = Variable(x.reshape((1,) + x.shape)), Variable(Y1_train[t]).to(torch.long), Variable(Y2_train[t]).to(torch.long)
          # b_x,b_y,b_y1 = data.cuda(),target1.cuda(),target2.cuda()
          b_x, b_y1, b_y2 = data, target1, target2
          b_x = b_x.float()
          model.train()
          optimizer.zero_grad()
          out,out1 = model(b_x)
          out = out.reshape((1,len(out)))
          out1 = out1.reshape((1, len(out1)))
          loss1 = loss_fn(out, b_y1)
          loss2 = loss_fn(out1,b_y2)
          loss = w1*loss1 + w2*loss2
          running_loss1 += loss1.item() * b_y1.size(0)+loss2.item() * b_y2.size(0)
          _, pred1 = torch.max(out, 1)
          _, pred2 = torch.max(out1, 1)
          num_correct1_train = (pred1 == b_y1).sum()
          num_correct2_train = (pred2 == b_y2).sum()
          # print(num_correct1, num_correct2)
          accuracy = (pred2 == b_y2).float().mean()
          running_acc1_train += num_correct1_train.item()
          running_acc2_train += num_correct2_train.item()
          # print(running_acc1, running_acc2)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      for t,x in tqdm(enumerate(X_test)):
          data, target1, target2 = Variable(x.reshape((1,) + x.shape)), Variable(Y1_test[t]).to(torch.long), Variable(Y2_test[t]).to(torch.long)
          # b_x,b_y,b_y1 = data.cuda(),target1.cuda(),target2.cuda()
          b_x, b_y1, b_y2 = data, target1, target2
          b_x = b_x.float()
          out,out1 = model(b_x)
          out = out.reshape((1,len(out)))
          out1 = out1.reshape((1, len(out1)))
          loss1 = loss_fn(out, b_y1)
          loss2 = loss_fn(out1,b_y2)
          running_loss2 += loss1.item() * b_y1.size(0)+loss2.item() * b_y2.size(0)
          _, pred1 = torch.max(out, 1)
          _, pred2 = torch.max(out1, 1)
          num_correct1_test = (pred1 == b_y1).sum()
          num_correct2_test = (pred2 == b_y2).sum()
          # print(num_correct1, num_correct2)
          accuracy = (pred2 == b_y2).float().mean()
          running_acc1_test += num_correct1_test.item()
          running_acc2_test += num_correct2_test.item()
          # print(running_acc1, running_acc2)

      print('Finish {} epoch, Loss_train: {:.6f}, Loss_test: {:.6f}, Acc1_train: {:.6f}, Acc2_train: {:.6f}, Acc1_test: {:.6f}, Acc2_test: {:.6f}'.format(
          epoch + 1, running_loss1 / len(X_train), running_loss2 / len(X_test), running_acc1_train / len(X_train), running_acc2_train / len(X_train), running_acc1_test / len(X_test), running_acc2_test / len(X_test)))
      logs += '\n'+str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))+'\tEpoch:[{}/{}]\t loss_train={:.5f}\t loss_test={:.5f}\t Acc1_train={:.5f}\t Acc2_train={:.5f}\t Acc1_test={:.5f}\t Acc2_test={:.5f}'.\
                  format(epoch+1 , Max_epoch, running_loss1 / len(X_train),running_loss2 / len(X_test), running_acc1_train / len(X_train), running_acc2_train / len(X_train), running_acc1_test / len(X_test), running_acc2_test / len(X_test)  )
      acclist.append([(running_acc1_test / len(X_test)+ running_acc2_test / len(X_test))/2,epoch])
      # torch.save(model,'/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))

  logs += '\n' + str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())) + '\tfinish training!'
  acclist.sort(reverse=True)
  best_epoch=acclist[0][1]
  # for epoch in range(Max_epoch):
  #   if epoch==best_epoch:
  #     model = torch.load('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))
  #     torch.save(model,'/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype)
  #   os.remove('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))
  # testtype = 'Ex1_12kDE_withoutDA'

  logs += '\nBest epoch:'+str(best_epoch+1)
  with open('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Logs/'+testtype+'.log','w+') as f:
      f.write(logs)
  torch.save(model, '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype)

choice 8


100%|██████████| 2700/2700 [00:22<00:00, 118.32it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:249: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:250: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:251: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:252: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().det

************************* epoch 1 *************************


1889it [01:09, 27.14it/s]
811it [00:01, 466.65it/s]


Finish 1 epoch, Loss_train: 1.881909, Loss_test: 1.888867, Acc1_train: 0.618317, Acc2_train: 0.629963, Acc1_test: 0.471023, Acc2_test: 0.773120
************************* epoch 2 *************************


...

1889it [01:02, 30.15it/s]
811it [00:01, 468.85it/s]


Finish 38 epoch, Loss_train: 1.136475, Loss_test: 1.211478, Acc1_train: 0.982530, Acc2_train: 0.983060, Acc1_test: 0.950678, Acc2_test: 0.939581
************************* epoch 39 *************************


1889it [01:02, 30.15it/s]
811it [00:01, 468.06it/s]


Finish 39 epoch, Loss_train: 1.133127, Loss_test: 1.204413, Acc1_train: 0.986236, Acc2_train: 0.983060, Acc1_test: 0.958076, Acc2_test: 0.940814
************************* epoch 40 *************************


20it [00:00, 30.58it/s]

In [ ]:
# 读取谷歌云盘的数据请求
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=False)

Drive not mounted, so nothing to flush and unmount.


ValueError: ignored

In [ ]:
import os, sys
path = os.getcwd()
datafiles = os.listdir('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis')
print(path)


/content
